## Clean Raw Data Scraped from Airbnb's Site

### Import packages

In [1]:
import os
import pandas as pd

### Read in the files from the search folder

In [2]:
results = os.listdir('search/')
#results

### Open the files and add the dates of the search performed to each file

In [3]:
open_file = pd.read_csv('search/'+results[0])
open_file['dates'] = str(results[0])
weekends = open_file.head(0)

In [4]:
for file in results: 
    open_file = pd.read_csv('search/'+file)
    open_file['dates'] = str(file)
    weekends = weekends.append(open_file)

In [5]:
weekends.reset_index(drop = True)
weekends.head(1)

,name,nightly cost,category,superhost,url,guests_rental,total cost,reviews,dates
0,NC5 The Joint!! Downtown Nashville Studio!,Price:$55 / night,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,6 guests · Studio · 2 beds · 1 bath,Price:$245 totalShow details,Price:$55 / nightPrice:$245 totalShow details,ci2020-12-17co2020-12-20.csv


### Create columns for checkin and checkout dates

In [6]:
# new data frame with split value columns 
dates = weekends['dates'].str.split('co', n = 1, expand = True)
weekends['checkin'] = dates[0] 
weekends['checkout'] = dates[1]
#weekends.head()

In [7]:
weekends['checkin'] = weekends['checkin'].str.replace('ci','')
weekends['checkout'] = weekends['checkout'].str.replace('.csv','')
weekends.drop('dates', axis=1, inplace=True)
#weekends.head()                                                    

In [8]:
weekends['checkin'] =  pd.to_datetime(weekends['checkin'], format='%Y/%m/%d')
weekends['checkout'] =  pd.to_datetime(weekends['checkout'], format='%Y/%m/%d')
#weekends.head()

### Clean the columns scrubbed from Airbnb's site

#### Create a column for the unique listing ID

In [9]:
weekends['ID'] = weekends['url'].str.partition('?check_in')[0]
weekends['ID'] = weekends['ID'].str.replace('/rooms/', '')
#weekends.head()

#### Create a column for the level of privacy (Entire, Private, Shared, etc.)

In [10]:
weekends['privacy'] = weekends['category'].str.partition(' ')[0]

# Combine types of stays to simplify categories
weekends['privacy'] = weekends['privacy'].str.replace('Hotel', 'Private')
weekends['privacy'] = weekends['privacy'].str.replace('Room', 'Private')
weekends['privacy'] = weekends['privacy'].str.replace('Resort', 'Private')
weekends['privacy'] = weekends['privacy'].str.replace('Campsite', 'Entire')
weekends['privacy'] = weekends['privacy'].str.replace('Camper/RV', 'Entire')
weekends['privacy'] = weekends['privacy'].str.replace('Tiny', 'Entire')
weekends['privacy'] = weekends['privacy'].str.replace('Barn', 'Entire')
weekends['privacy'] = weekends['privacy'].str.replace('Farm', 'Entire')
weekends['privacy'] = weekends['privacy'].str.replace('Bus', 'Entire')
weekends['privacy'] = weekends['privacy'].str.replace('Tent', 'Entire')
weekends['privacy'] = weekends['privacy'].str.replace('Hostel', 'Shared')

#weekends['privacy'].value_counts()

#### Create a column for the number of guests 

In [11]:
weekends['guests'] = weekends['guests_rental'].str.partition(' · ')[0]
#weekends.head()

In [12]:
weekends['guest_rental2'] = weekends['guests_rental'].str.partition(' · ')[2]
#weekends.head()

#### Create a column for the number of bedrooms

In [13]:
weekends['bedroom'] = weekends['guest_rental2'].str.partition(' · ')[0] 
#weekends.head()

In [14]:
weekends['guest_rental3'] = weekends['guest_rental2'].str.partition(' · ')[2]
#weekends.head()

#### Create a column for the number of beds

In [15]:
weekends['beds'] = weekends['guest_rental3'].str.partition(' · ')[0] 
#weekends.head()

In [16]:
weekends['bath'] = weekends['guest_rental3'].str.partition(' · ')[2]
#weekends.head()

#### Drop original columns that have been cleaned

In [17]:
weekends.drop(columns = ['guests_rental', 'guest_rental2', 'guest_rental3'], axis=1, inplace=True)
#weekends.head()

#### Create columns for the prices associated with each listing

In [18]:
weekends['not_discounted'] = weekends['nightly cost'].str.partition('Previous price:')[0]
#weekends.head()

In [19]:
weekends['test'] = weekends['nightly cost'].str.partition('Previous price:')[2]
#weekends.head()

In [20]:
weekends['discounted'] = weekends['test'].str.partition(':')[0]
#weekends.head()

In [21]:
weekends['discounted'] = weekends['test'].str.partition(':')[2]
#weekends.head()

In [22]:
weekends['previous_price'] = weekends['test'].str.partition(':')[0]
#weekends.head()

In [23]:
weekends.drop(columns=['test', 'nightly cost'], axis=1, inplace=True)
#weekends.head()

In [24]:
weekends['not_discounted'] = weekends['not_discounted'].str.replace('Price:', '') 
weekends['not_discounted'] = weekends['not_discounted'].str.replace(' / night', '')

#weekends.head()

In [25]:
weekends['discounted'] = weekends['discounted'].str.replace(' / night', '')
#weekends.head()

In [26]:
weekends['previous_price'] = weekends['previous_price'].str.replace('Discounted price', '')
#weekends.head()

In [27]:
weekends['current_nightly'] = weekends['not_discounted'] + weekends['discounted']
#weekends.head()

#### Remove redundant words from values in columns with clear descriptions

In [28]:
weekends['guests'] = weekends['guests'].str.replace(' guests', '')
#weekends.head()

In [29]:
weekends['guests'] = weekends['guests'].str.replace(' guest', '')
#weekends.head(25)

In [30]:
weekends['beds'] = weekends['beds'].str.replace(' beds', '')
weekends['beds'] = weekends['beds'].str.replace(' bed', '')
#weekends.head()

In [31]:
weekends['total cost'] = weekends['total cost'].str.replace('Price:', '')
weekends['total cost'] = weekends['total cost'].str.replace('totalShow details', '')
#weekends.head()

#### Create columns for number of reviews and average stars

In [32]:
weekends['review_test'] = weekends['reviews'].str.partition('Rating')[2]
#weekends.head()

In [33]:
weekends['avg_stars'] = weekends['review_test'].str.partition(' out of')[0]
#weekends.head()

In [34]:
weekends['review_count2'] = weekends['review_test'].str.partition('(')[2]
#weekends.head()

In [35]:
weekends['review_count'] = weekends['review_count2'].str.partition(')')[0]
#weekends.head()

In [36]:
weekends.drop(columns = ['review_count2', 'review_test', 'reviews'], axis=1, inplace=True)
#weekends.head()

#### Drop original columns that have been cleaned

In [37]:
weekends = weekends[['ID', 'name', 'category', 'privacy', 'superhost', 'url', 'avg_stars', 'review_count', 'guests','beds', 'bedroom', 'bath', 'checkin', 'checkout', 'previous_price', 'discounted', 'not_discounted', 'current_nightly', 'total cost']]
weekends.head(2)

,ID,name,category,privacy,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost
0,46779266,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,Entire,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,$55,$245
1,35440277,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,Entire,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,$122,$58,,$58,$329


#### Check data types before performing calculations on prices

In [38]:
weekends.dtypes

ID                         object
name                       object
category                   object
privacy                    object
superhost                  object
url                        object
avg_stars                  object
review_count               object
guests                     object
beds                       object
bedroom                    object
bath                       object
checkin            datetime64[ns]
checkout           datetime64[ns]
previous_price             object
discounted                 object
not_discounted             object
current_nightly            object
total cost                 object
dtype: object

#### Remove extraneous characters from price columns and convert to integers

In [39]:
weekends['current_nightly'] = weekends['current_nightly'].astype(str)

weekends['current_nightly'] = weekends['current_nightly'].str.replace('$','')
weekends['current_nightly'] = weekends['current_nightly'].str.replace(',','')
weekends['current_nightly'] = weekends['current_nightly'].str.replace('nan', '0')

weekends['current_nightly'] = weekends['current_nightly'].astype(int)

#weekends.head()

In [40]:
weekends['total cost'] = weekends['total cost'].astype(str)

weekends['total cost'] = weekends['total cost'].str.replace('$','')
weekends['total cost'] = weekends['total cost'].str.replace(',','')
weekends['total cost'] = weekends['total cost'].str.replace(' total','')
weekends['total cost'] = weekends['total cost'].str.replace('nan', '0')

weekends['total cost'] = weekends['total cost'].astype(int)

#weekends.head()

In [41]:
weekends['previous_price'] = weekends['previous_price'].astype(str)

weekends['previous_price'] = weekends['previous_price'].str.replace('$','')
weekends['previous_price'] = weekends['previous_price'].str.replace(',','')
weekends['previous_price'] = weekends['previous_price'].str.replace(' total','')
weekends['previous_price'] = weekends['previous_price'].str.replace('nan', '0')

#weekends.head()

In [42]:
weekends['discounted'] = weekends['discounted'].astype(str)

weekends['discounted'] = weekends['discounted'].str.replace('$','')
weekends['discounted'] = weekends['discounted'].str.replace(',','')
weekends['discounted'] = weekends['discounted'].str.replace(' total','')
weekends['discounted'] = weekends['discounted'].str.replace('nan', '0')

#weekends.head()

In [43]:
weekends['not_discounted'] = weekends['not_discounted'].astype(str)

weekends['not_discounted'] = weekends['not_discounted'].str.replace('$','')
weekends['not_discounted'] = weekends['not_discounted'].str.replace(',','')
weekends['not_discounted'] = weekends['not_discounted'].str.replace(' total','')
weekends['not_discounted'] = weekends['not_discounted'].str.replace('nan', '0')

#weekends.head()

#### Calculate the room fee (nightly rate * # of nights staying) and the additional fees

In [44]:
weekends['room_fee'] = weekends['current_nightly'] * 3 
#weekends.head()

In [45]:
weekends['taxes_fees_cleaning'] = weekends['total cost'] - weekends['room_fee']
#weekends.head()

### Drop empty and duplicate rows

In [46]:
weekends.dropna(subset=['name', 'url'], inplace=True)
weekends.shape

(64223, 21)

In [47]:
weekends.drop_duplicates(subset=['ID', 'checkin'], keep='first', inplace=True)
weekends.shape

(58674, 21)

### Calculate occupancy rate for each listing

In [48]:
weekends['occupancy count'] = weekends.groupby('ID')['ID'].transform('count')

In [49]:
weekends['occupancy rate'] = (26 - weekends['occupancy count']) / 26

### Save to a csv

In [157]:
weekends.to_csv('data_for_viz/weekends.csv')